---
title: "Data Cleaning"
format: 
  html:
    embed-resources: true
---

In [1]:
# imports 
import pandas as pd
import numpy as np

## Poverty Data Cleaning

In [2]:
# read in data
poverty_df = pd.read_csv("../data/raw/Poverty_Data_2022.csv")

# check for NAs
print("\n-------------\nINITIAL NA COUNT:\n-------------\n")
print(poverty_df.isna().sum())

# Rename columns for simplicity
poverty_df = poverty_df.rename(columns={
    "State Postal Code": "state_name",
    "State FIPS code": "state_fips",
    "Estimated Population 5-17": "pop_students",
    "Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder": "pop_students_poverty"
})

# Clean column names: lowercase and replace spaces with underscores
poverty_df.columns = poverty_df.columns.str.lower().str.replace(" ", "_")

# Remove commas from numeric columns and convert to float
poverty_df["pop_students_poverty"] = poverty_df["pop_students_poverty"].astype(str).str.replace(",", "").astype(float)
poverty_df["pop_students"] = poverty_df["pop_students"].astype(str).str.replace(",", "").astype(float)

# create a student poverty ratio variable 
poverty_df["student_poverty_ratio"] = poverty_df["pop_students_poverty"] / poverty_df["pop_students"]
poverty_df["student_poverty_ratio"] = poverty_df["student_poverty_ratio"].fillna(0)  # Replace NaN (if any) with 0

# create leaid variable using fips code and district id
poverty_df["leaid"] = poverty_df["state_fips_code"].astype(str).str.zfill(1) + poverty_df["district_id"].astype(str).str.zfill(5)
poverty_df["leaid"] = poverty_df["leaid"].astype(int)

# Drop unnecessary columns
poverty_df = poverty_df[["leaid","student_poverty_ratio"]]

# check for NAs again
print("\n-------------\nFINAL NA COUNT:\n-------------\n")
print(poverty_df.isna().sum())

# Print shape and show df
print("\n-------------\nSHAPE:\n-------------\n")
print(poverty_df.shape)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
poverty_df.head()


-------------
INITIAL NA COUNT:
-------------

State Postal Code                                                                                        0
State FIPS Code                                                                                          0
District ID                                                                                              0
Name                                                                                                     0
Estimated Total Population                                                                               0
Estimated Population 5-17                                                                                0
Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder    0
dtype: int64

-------------
FINAL NA COUNT:
-------------

leaid                    0
student_poverty_ratio    0
dtype: int64

-------------
SHAPE:
-------------

(13144, 2)

-------------
GLIMPSE DF:
--

,leaid,student_poverty_ratio
0,100190,0.105866
1,100005,0.215959
2,100030,0.234857
3,100060,0.282290
4,100090,0.374703


## Absentee Demographics Cleaning

In [3]:
# Dictionary mapping old column names to new column names
rename_dict = {
    "AM7_T": "Am_Indian_Total",
    "AM7_F": "Am_Indian_Female",
    "AM7_M": "Am_Indian_Male",
    "AS7_T": "Asian_Total",
    "AS7_F": "Asian_Female",
    "AS7_M": "Asian_Male",
    "PI7_T": "Pac_Islander_Total",
    "PI7_F": "Pac_Islander_Female",
    "PI7_M": "Pac_Islander_Male",
    "BL7_T": "Black_Total",
    "BL7_F": "Black_Female",
    "BL7_M": "Black_Male",
    "HI7_T": "Hispanic_Total",
    "HI7_F": "Hispanic_Female",
    "HI7_M": "Hispanic_Male",
    "MU7_T": "Multiracial_Total",
    "MU7_F": "Multiracial_Female",
    "MU7_M": "Multiracial_Male",
    "WH7_T": "White_Total",
    "WH7_F": "White_Female",
    "WH7_M": "White_Male",
    "WDIS_T": "Disabilities_Total",
    "WDIS_F": "Disabilities_Female",
    "WDIS_M": "Disabilities_Male",
    "DISAB504STAT_T": "Sec504_Total",
    "DISAB504STAT_F": "Sec504_Female",
    "DISAB504STAT_M": "Sec504_Male",
    "ECODIS_T": "Econ_Disadv_Total",
    "ECODIS_F": "Econ_Disadv_Female",
    "ECODIS_M": "Econ_Disadv_Male",
    "LEP_T": "Eng_Learner_Total",
    "LEP_F": "Eng_Learner_Female",
    "LEP_M": "Eng_Learner_Male",
    "HOM_T": "Homeless_Total",
    "HOM_F": "Homeless_Female",
    "HOM_M": "Homeless_Male",
}

state_abbrev = {
    "ALABAMA": "AL", "ALASKA": "AK", "ARIZONA": "AZ", "ARKANSAS": "AR", "CALIFORNIA": "CA",
    "COLORADO": "CO", "CONNECTICUT": "CT", "DELAWARE": "DE", "FLORIDA": "FL", "GEORGIA": "GA",
    "HAWAII": "HI", "IDAHO": "ID", "ILLINOIS": "IL", "INDIANA": "IN", "IOWA": "IA",
    "KANSAS": "KS", "KENTUCKY": "KY", "LOUISIANA": "LA", "MAINE": "ME", "MARYLAND": "MD",
    "MASSACHUSETTS": "MA", "MICHIGAN": "MI", "MINNESOTA": "MN", "MISSISSIPPI": "MS", "MISSOURI": "MO",
    "MONTANA": "MT", "NEBRASKA": "NE", "NEVADA": "NV", "NEW HAMPSHIRE": "NH", "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM", "NEW YORK": "NY", "NORTH CAROLINA": "NC", "NORTH DAKOTA": "ND", "OHIO": "OH",
    "OKLAHOMA": "OK", "OREGON": "OR", "PENNSYLVANIA": "PA", "RHODE ISLAND": "RI", "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD", "TENNESSEE": "TN", "TEXAS": "TX", "UTAH": "UT", "VERMONT": "VT",
    "VIRGINIA": "VA", "WASHINGTON": "WA", "WEST VIRGINIA": "WV", "WISCONSIN": "WI", "WYOMING": "WY"
}

# read in Absentee Demographic Data
absentee_demo_df = pd.read_csv("../data/raw/Chronic_Absenteeism_Demographics_2022.csv")

# check for NAs
print("\n-------------\nINITIAL NA COUNT:\n-------------\n")
print(absentee_demo_df.isna().sum())

# drop unecessary columns
absentee_demo_df = absentee_demo_df.drop(columns=["SCHOOL_YEAR", "DATA_GROUP_ID", "TEXT_VALUE", "STATE_NAME", "LEA_NAME"])

# Clean the absentee_demo_df DataFrame
absentee_demo_df = absentee_demo_df[~absentee_demo_df["SUBGROUP"].isin(["ALLLEA"])]
absentee_demo_df["CHARACTERISTICS"] = absentee_demo_df["CHARACTERISTICS"].fillna("T")

# Combine "SUBGROUP" and "CHARACTERISTICS" into a new "race_and_gender" column
absentee_demo_df["race_and_gender"] = absentee_demo_df["SUBGROUP"].astype(str) + "_" + absentee_demo_df["CHARACTERISTICS"].astype(str)

# Drop the original "SUBGROUP" and "CHARACTERISTICS" columns
absentee_demo_df = absentee_demo_df.drop(columns=["SUBGROUP", "CHARACTERISTICS"])

# Pivot the DataFrame to make it wide
df_wide = absentee_demo_df.pivot_table(index=["LEAID"], 
                                       columns=["race_and_gender"], 
                                       values="NUMERIC_VALUE").reset_index()
df_wide.columns.name = None

# fill NAs with 0
df_wide = df_wide.fillna(0)

# Rename columns to defined dict above
df_wide = df_wide.rename(columns=rename_dict)

# Clean column names: lowercase and replace spaces with underscores
df_wide.columns = df_wide.columns.str.lower().str.replace(" ", "_")

# check for NAs again
print("\n-------------\nFINAL NA COUNT:\n-------------\n")
print(df_wide.isna().sum())

# Print shape and show df
print("\n-------------\nSHAPE:\n-------------\n")
print(df_wide.shape)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
df_wide.head()


-------------
INITIAL NA COUNT:
-------------

SCHOOL_YEAR             0
STATE_NAME              0
LEAID                   0
LEA_NAME                0
DATA_GROUP_ID           0
NUMERIC_VALUE           0
TEXT_VALUE              0
SUBGROUP                0
CHARACTERISTICS    176915
dtype: int64

-------------
FINAL NA COUNT:
-------------

leaid                  0
am_indian_female       0
am_indian_male         0
am_indian_total        0
asian_female           0
asian_male             0
asian_total            0
black_female           0
black_male             0
black_total            0
sec504_female          0
sec504_male            0
sec504_total           0
econ_disadv_female     0
econ_disadv_male       0
econ_disadv_total      0
hispanic_female        0
hispanic_male          0
hispanic_total         0
homeless_female        0
homeless_male          0
homeless_total         0
eng_learner_female     0
eng_learner_male       0
eng_learner_total      0
multiracial_female     0
multiraci

,leaid,am_indian_female,am_indian_male,am_indian_total,asian_female,asian_male,asian_total,black_female,black_male,black_total,...,multiracial_total,pac_islander_female,pac_islander_male,pac_islander_total,disabilities_female,disabilities_male,disabilities_total,white_female,white_male,white_total
0,100005,4.0,2.0,6.0,2.0,1.0,3.0,24.0,24.0,48.0,...,70.0,1.0,0.0,1.0,82.0,151.0,233.0,315.0,337.0,652.0
1,100006,2.0,0.0,2.0,1.0,1.0,2.0,6.0,13.0,19.0,...,20.0,0.0,0.0,0.0,97.0,141.0,238.0,395.0,379.0,774.0
2,100007,0.0,2.0,2.0,43.0,47.0,90.0,233.0,186.0,419.0,...,87.0,0.0,1.0,1.0,107.0,168.0,275.0,327.0,313.0,640.0
3,100008,4.0,3.0,7.0,39.0,33.0,72.0,239.0,196.0,435.0,...,134.0,3.0,1.0,4.0,159.0,247.0,406.0,502.0,493.0,995.0
4,100011,0.0,0.0,0.0,3.0,0.0,3.0,77.0,79.0,156.0,...,13.0,0.0,0.0,0.0,36.0,90.0,126.0,136.0,184.0,320.0


## Absentee Totals Cleaning

In [4]:
# read in data
absentee_total_df = pd.read_csv("../data/raw/Chronic_Absenteeism_Total_2022.csv")

# check for NAs
print("\n-------------\nINITIAL NA COUNT:\n-------------\n")
print(absentee_total_df.isna().sum())

# drop any rows with missing numeric values
absentee_total_df = absentee_total_df.dropna(subset=["NUMERIC_VALUE"])

# Drop unneeded columns from absentee data
absentee_total_df = absentee_total_df.drop(columns=["SCHOOL_YEAR", "DATA_GROUP_ID", "TEXT_VALUE", "SUBGROUP"])

# Rename columns
absentee_total_df = absentee_total_df.rename(columns={
  "DENOMINATOR": "total_students", 
  "NUMERATOR": "num_chronically_absent_students", 
  "NUMERIC_VALUE":"chronic_absentee_ratio"})

# Convert full state names to abbreviations
absentee_total_df["STATE_NAME"] = absentee_total_df["STATE_NAME"].str.upper().map(state_abbrev)
# drop any rows with missing state names (BIA and Puerto Rico schools)
absentee_total_df = absentee_total_df.dropna(subset=["STATE_NAME"])

# create a ratio column
absentee_total_df["chronic_absentee_ratio"] = round(absentee_total_df["chronic_absentee_ratio"] / 100, 3)

# Clean column names: lowercase and replace spaces with underscores
absentee_total_df.columns = absentee_total_df.columns.str.lower().str.replace(" ", "_")

# check for NAs again
print("\n-------------\nFINAL NA COUNT:\n-------------\n")
print(absentee_total_df.isna().sum())

# Print shape and show df
print("\n-------------\nSHAPE:\n-------------\n")
print(absentee_total_df.shape)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
absentee_total_df.head()


-------------
INITIAL NA COUNT:
-------------

SCHOOL_YEAR       0
STATE_NAME        0
LEAID             0
LEA_NAME          0
DATA_GROUP_ID     0
DENOMINATOR       0
NUMERATOR         0
TEXT_VALUE        0
NUMERIC_VALUE    14
SUBGROUP          0
dtype: int64

-------------
FINAL NA COUNT:
-------------

state_name                         0
leaid                              0
lea_name                           0
total_students                     0
num_chronically_absent_students    0
chronic_absentee_ratio             0
dtype: int64

-------------
SHAPE:
-------------

(17747, 6)

-------------
GLIMPSE DF:
-------------



,state_name,leaid,lea_name,total_students,num_chronically_absent_students,chronic_absentee_ratio
0,AL,100005,Albertville City,5767,1320,0.229
1,AL,100006,Marshall County,5717,974,0.170
2,AL,100007,Hoover City,13300,1398,0.105
3,AL,100008,Madison City,12187,1855,0.152
4,AL,100011,Leeds City,2081,572,0.275


## School Dempographics Cleaning

In [5]:
# read in the data
demographic_df = pd.read_csv("../data/raw/demographic.csv")

# drop rows that have data coded as not applicable
demographic_df = demographic_df[~demographic_df.apply(lambda row: row.astype(str).str.contains("†")).any(axis=1)]

print("\n-------------\nINITIAL NA COUNT:\n-------------\n")
print(demographic_df.isna().sum())

demographic_df.head()

# replace column names 
new_col_names = [
    "lea_name",
    "state_name",
    "american_indian_alaska_native",
    "asian_pacific_islander",
    "hispanic",
    "black_african_american",
    "white",
    "native_hawaiian_pacific_islander",
    "total_race_ethnicity",
    "leaid"
]
demographic_df.columns = new_col_names
demographic_df = demographic_df.sort_values(by="leaid", ascending=True)

# convert columns to numeric, filling blanks with 0
for col in new_col_names[2:]:
  demographic_df[col] = pd.to_numeric(demographic_df[col], errors='coerce').fillna(0)

# crate new columns for percentages
for col in new_col_names[2:8]:
  demographic_df[col + "_pct"] = round(demographic_df[col] / demographic_df["total_race_ethnicity"], 4)

# drop all columns except leaid and the new percentage columns
demographic_df = demographic_df.drop(columns=new_col_names[:9])

# drop any na and sort by leaid
demographic_df = demographic_df.dropna()
demographic_df = demographic_df.sort_values(by="leaid", ascending=True).reset_index(drop=True)

# check for NAs again
print("\n-------------\nFINAL NA COUNT:\n-------------\n")
print(demographic_df.isna().sum())

# Print shape and show df
print("\n-------------\nSHAPE:\n-------------\n")
print(demographic_df.shape)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
demographic_df.head()


-------------
INITIAL NA COUNT:
-------------

Agency Name                                                        0
State Name [District] Latest available year                        0
American Indian/Alaska Native Students [District] 2022-23          0
Asian or Asian/Pacific Islander Students [District] 2022-23        0
Hispanic Students [District] 2022-23                               0
Black or African American Students [District] 2022-23              0
White Students [District] 2022-23                                  0
Nat. Hawaiian or Other Pacific Isl. Students [District] 2022-23    0
Total Race/Ethnicity [District] 2022-23                            0
Agency ID - NCES Assigned [District] Latest available year         0
dtype: int64

-------------
FINAL NA COUNT:
-------------

leaid                                   0
american_indian_alaska_native_pct       0
asian_pacific_islander_pct              0
hispanic_pct                            0
black_african_american_pct         

,leaid,american_indian_alaska_native_pct,asian_pacific_islander_pct,hispanic_pct,black_african_american_pct,white_pct,native_hawaiian_pacific_islander_pct
0,100005,0.0051,0.0047,0.5637,0.0454,0.3563,0.0003
1,100006,0.0055,0.0042,0.2840,0.0145,0.6787,0.0012
2,100007,0.0012,0.0744,0.0928,0.2298,0.5542,0.0009
3,100008,0.0046,0.0895,0.0844,0.1894,0.5667,0.0016
4,100011,0.0009,0.0054,0.1502,0.2555,0.5677,0.0000


## Financial Data Cleaning

In [6]:
financial_df = pd.read_csv("../data/raw/Financial_Data_2022.csv", dtype={"LEAID": str})

# check for NAs
print("\n-------------\nINITIAL NA COUNT:\n-------------\n")
print(financial_df.isna().sum())

# coerce leaid to numeric drop bad rows
financial_df["leaid"] = pd.to_numeric(financial_df["LEAID"], errors="coerce")

# Drop rows where LEAID is NaN
financial_df = financial_df.dropna(subset=["leaid"])

# Keep only certain columns in the larger financial dataframe
financial_df = financial_df[["leaid", "TFEDREV", "TSTREV", "TLOCREV", "TOTALEXP", "Z33", "Z34"]]
financial_df = financial_df.rename(columns={
  "TFEDREV": "total_federal_revenue", 
  "TSTREV": "total_state_revenue",
  "TLOCREV": "total_local_revenue", 
  "TOTALEXP": "total_expenditures",
  "Z33": "total_salaries", 
  "Z34": "total_employee_benefits"})

# Keep only rows where all values are >= 0
financial_df = financial_df[(financial_df >= 0).all(axis=1)]

# check na again
print("\n-------------\nFINAL NA COUNT:\n-------------\n")
print(financial_df.isna().sum())

# Print shape and show df
print("\n-------------\nSHAPE:\n-------------\n")
print(financial_df.shape)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
financial_df.head()


-------------
INITIAL NA COUNT:
-------------

LEAID        0
CENSUSID     0
PID6         0
UNIT_TYPE    0
FIPST        0
            ..
FL_AE4G      0
FL_AE5       0
FL_AE6       0
FL_AE7       0
FL_AE8       0
Length: 354, dtype: int64

-------------
FINAL NA COUNT:
-------------

leaid                      0
total_federal_revenue      0
total_state_revenue        0
total_local_revenue        0
total_expenditures         0
total_salaries             0
total_employee_benefits    0
dtype: int64

-------------
SHAPE:
-------------

(18185, 7)

-------------
GLIMPSE DF:
-------------



,leaid,total_federal_revenue,total_state_revenue,total_local_revenue,total_expenditures,total_salaries,total_employee_benefits
1,100005.0,13132000,42566000,18900000,65734000,18998000,11412000
2,100006.0,24404000,45327000,18823000,87804000,22575000,14775000
3,100007.0,18644000,87393000,113032000,208943000,73899000,40534000
4,100008.0,14037000,85179000,78543000,173520000,49853000,27452000
7,100011.0,5158000,15284000,11143000,28267000,9146000,5064000


## Merging Data 

In [7]:
# merge data. 
merge1_df = absentee_total_df.merge(df_wide, on = "leaid", how="inner")
merge2_df = merge1_df.merge(demographic_df, on = "leaid", how="inner")
merge3_df = merge2_df.merge(poverty_df, on = "leaid", how="inner")
final_df = merge3_df.merge(financial_df, on = "leaid", how="inner")

# standardize all column names to lowercase
final_df.columns = final_df.columns.str.lower()

print("\n-------------\nFINAL NA COUNT:\n-------------\n")
print(final_df.isna().sum())

# Print shape and show df
print("\n-------------\nSHAPE:\n-------------\n")
print(final_df.shape)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
final_df.head()


-------------
FINAL NA COUNT:
-------------

state_name                              0
leaid                                   0
lea_name                                0
total_students                          0
num_chronically_absent_students         0
chronic_absentee_ratio                  0
am_indian_female                        0
am_indian_male                          0
am_indian_total                         0
asian_female                            0
asian_male                              0
asian_total                             0
black_female                            0
black_male                              0
black_total                             0
sec504_female                           0
sec504_male                             0
sec504_total                            0
econ_disadv_female                      0
econ_disadv_male                        0
econ_disadv_total                       0
hispanic_female                         0
hispanic_male                 

,state_name,leaid,lea_name,total_students,num_chronically_absent_students,chronic_absentee_ratio,am_indian_female,am_indian_male,am_indian_total,asian_female,...,black_african_american_pct,white_pct,native_hawaiian_pacific_islander_pct,student_poverty_ratio,total_federal_revenue,total_state_revenue,total_local_revenue,total_expenditures,total_salaries,total_employee_benefits
0,AL,100005,Albertville City,5767,1320,0.229,4.0,2.0,6.0,2.0,...,0.0454,0.3563,0.0003,0.215959,13132000,42566000,18900000,65734000,18998000,11412000
1,AL,100006,Marshall County,5717,974,0.170,2.0,0.0,2.0,1.0,...,0.0145,0.6787,0.0012,0.196512,24404000,45327000,18823000,87804000,22575000,14775000
2,AL,100007,Hoover City,13300,1398,0.105,0.0,2.0,2.0,43.0,...,0.2298,0.5542,0.0009,0.073843,18644000,87393000,113032000,208943000,73899000,40534000
3,AL,100008,Madison City,12187,1855,0.152,4.0,3.0,7.0,39.0,...,0.1894,0.5667,0.0016,0.056285,14037000,85179000,78543000,173520000,49853000,27452000
4,AL,100011,Leeds City,2081,572,0.275,0.0,0.0,0.0,3.0,...,0.2555,0.5677,0.0000,0.169442,5158000,15284000,11143000,28267000,9146000,5064000


## Data Extraction

In [8]:
# create other important features
final_df['expenditures_per_student'] = final_df['total_expenditures']/final_df['total_students']

# CREATE TARGETS 

# Create binary targets: 1 = high absenteeism
mean_absentee = final_df['chronic_absentee_ratio'].mean()
final_df['high_absenteeism'] = (final_df['chronic_absentee_ratio'] > mean_absentee).astype(int)

# Make another one for Dept of Ed Statistic
final_df['high_absenteeism_doe'] = (final_df['chronic_absentee_ratio'] > 0.28).astype(int)

# Create a multiclass for low middle and high absenteeism
final_df['absenteeism_class'] = pd.qcut(
    final_df['chronic_absentee_ratio'],
    q=3,
    labels=[1, 2, 3] # 1 = low, 2 = middle, 3 = high
)

# Write the final DataFrame to a CSV file
final_df.to_csv("../data/clean/cleaned_school_data.csv", index = False)

print("\n-------------\nGLIMPSE DF:\n-------------\n")
final_df.head()


-------------
GLIMPSE DF:
-------------



,state_name,leaid,lea_name,total_students,num_chronically_absent_students,chronic_absentee_ratio,am_indian_female,am_indian_male,am_indian_total,asian_female,...,total_federal_revenue,total_state_revenue,total_local_revenue,total_expenditures,total_salaries,total_employee_benefits,expenditures_per_student,high_absenteeism,high_absenteeism_doe,absenteeism_class
0,AL,100005,Albertville City,5767,1320,0.229,4.0,2.0,6.0,2.0,...,13132000,42566000,18900000,65734000,18998000,11412000,11398.300676,0,0,2
1,AL,100006,Marshall County,5717,974,0.170,2.0,0.0,2.0,1.0,...,24404000,45327000,18823000,87804000,22575000,14775000,15358.404758,0,0,2
2,AL,100007,Hoover City,13300,1398,0.105,0.0,2.0,2.0,43.0,...,18644000,87393000,113032000,208943000,73899000,40534000,15710.000000,0,0,1
3,AL,100008,Madison City,12187,1855,0.152,4.0,3.0,7.0,39.0,...,14037000,85179000,78543000,173520000,49853000,27452000,14238.122590,0,0,1
4,AL,100011,Leeds City,2081,572,0.275,0.0,0.0,0.0,3.0,...,5158000,15284000,11143000,28267000,9146000,5064000,13583.373378,1,0,3
